# 준비작업

ToDo List
1. __발급불가 판정 나는 거 pass하는 코드__
1. 사건번호로 채무자키 찾을 때 관할법원/관련사건법원 까지 함께 살피기(여러개 나오는 경우 공백으로 두기 때문에 수작업을 해야할 뿐 잘못된 키가 불려오지는 않을 것)
2. 1을 위해서 관할법원 정식명칭으로 맵핑하는 작업(일단은 지방> 지방법원 만 적용중. 현재 광주 아니면 부산이라 문제 없음)
3. 신규등록해야 하는 건 선별해서 법조치에, 기등록건은 법조치키를 법비용에
4. 대상자구분 확인하기
5. (전계좌, 부분계좌, 단일계좌 확인해야....)
6. 법조치 기등록건 구분 및 법조치키 불러오기

### import
- 이행권고 출력 및 판결문 목록에 등재 (화해권고는 다른 사람이 출력중)
- 승계는 수작업(발급당사자는 원고만, 납부인)

In [1]:
import pyautogui as au
import time
import pandas as pd
import re
from os.path import join
import os
import numpy as np
from selenium.webdriver.chrome.service import Service
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.alert import Alert
from selenium.common.exceptions import NoAlertPresentException, NoSuchWindowException # NoSuchElementException, UnexpectedAlertPresentException, WebDriverException,
import random
from file_function import read_filelist
from datetime import date
from datetime import timedelta
import traceback
# 정렬순서 : 확정일자(내림), 법원(올림), 사건번호(내림)

dtype = {'채무자키':str, '계좌키':str, '가상계좌번호':str, '법조치키':str}

### 함수, 전역변수

In [2]:
##################################
# 사건번호 눌러보고 집행문 발급 가능한 것만으로 목록작성 하도록!!!!!!!!!
lastday = "240526"
법원인증서="헬프자산관리대부[주]()001102620170619111002243" # 인증서 여러개고, 법원인증서가 첫번째가 아닌 경우 지정해줘야 함

이행권고출력 = False # 이행권고 출력할지 말지 (240513이후 확정건부터 박근혜과장님이 출력)
##################################
excel_dir = join(r"D:\4.관리\전자소송", lastday)
excel_file = "집행문신청_" + lastday
path_excel = join(excel_dir, excel_file+".xlsx")
bank = "신한은행"
driver, wait = None, None

def 파일명(폴더:str, 키워드:str) :
    file_list = os.listdir(폴더)
    account_filename = [file for file in file_list if re.search(키워드, file)]
    if len(account_filename) == 1 :
        return join(폴더,account_filename[0])
    else :
        print("해당 키워드가 들어간 파일이 하나가 아닙니다.")

# pause
def lwt() : time.sleep(2.5)
def mwt() : time.sleep(1)
def swt() : time.sleep(random.uniform(0.1,0.4))

def set_chrome_driver():
    chrome_options = webdriver.ChromeOptions()
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
    return driver

def con_scourt() :
    driver = set_chrome_driver()
    driver.get("https://ecfs.scourt.go.kr/ecf/index.jsp")
    wait = WebDriverWait(driver, 30) 
    wait.until(EC.element_to_be_clickable((By.ID, 'userId1')))
    mwt()
    return driver, wait

def certify() :
    elem = wait.until(EC.element_to_be_clickable((By.ID, "xwup_certselect_tek_input1")))
    mwt()
    # 여러 인증서중 법원용 인증서 선택
    driver.find_element(By.XPATH, '//*[@id="xwup_cert_table"]/table/tbody/tr/td/div[text()="' + 법원인증서 + '"]').click()
    swt()
    elem.send_keys("help8700765!")
    swt()
    driver.find_element(By.ID, 'xwup_OkButton').click() # 인증서 확인
    mwt()

def position_new_win():
    position = driver.get_window_position()
    x = position.get('x')
    y = position.get('y')
    return x, y

def login() : 
    # 로그인(id) # 셀레니움 안 돼서.. 
    x, y = position_new_win()
    au.click(x+100,y+315) # 305 > 315 #240402
    time.sleep(0.2)
    au.write("help")
    au.press("num2")
    au.press("num2")
    au.write("55")
    driver.find_element(By.ID, "userId1").send_keys(Keys.ENTER)
    # 로그인(인증서)
    certify()
    lwt()
############ 공통 사용 #############

def close_all_popups_and_refresh(driver):
    try:
        # 모든 팝업 창 닫기
        if len(driver.window_handles) > 1 : 
            for handle in driver.window_handles[1:]:
                driver.switch_to.window(handle)
                driver.close()
        
        # 첫 번째 창으로 돌아가기
        driver.switch_to.window(driver.window_handles[0])
        
    except Exception as e:
        print(f"팝업 창을 닫는 동안 오류가 발생했습니다: {e}")

    # 브라우저 새로고침
    driver.refresh()
    lwt()


def apply_or_print(AorP : int) :
    "apply_or_print : 신청 : 1, 발급 : 2"
    wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="gnb"]/ul/li[4]/a'))).send_keys(Keys.ENTER) # 제증명
    driver.find_element(By.XPATH, '//*[@id="tmenu04"]/div/ul/li['+str(AorP)+']/a').send_keys(Keys.ENTER) # 제증명신청/발급
    swt()
    
def 이행권고() : # 제증명 > 발급
    wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="gnb"]/ul/li[3]/a'))).send_keys(Keys.ENTER) # 열람/발급
    driver.find_element(By.XPATH, '//*[@id="tmenu03"]/div/ul/li[2]/a').send_keys(Keys.ENTER) # 정(등)본발급
    swt()

def select_court_event(court, event) :
    """법원 및 사건번호 입력"""
    # 법원
    Select(driver.find_element(By.ID, 'cbxBubCd')).select_by_visible_text(court) # 집행문 선택
    swt()
    
    # 사건번호 선택
    event = re.search("(\d{4,4})([가-힣]+)(\d+)", event)
    year = event[1]
    sign = event[2]
    sn = event[3]
    Select(driver.find_element(By.XPATH, '//select[@title="사건연도"]')).select_by_value(year) # 연도 선택
    Select(driver.find_element(By.XPATH, '//select[@title="사건구분"]')).select_by_visible_text(sign)  # 사건구분 선택
    driver.find_element(By.XPATH, '//input[contains(@title,"사건일련번호")]').send_keys(sn)  # 사건일련번호 선택
    swt()
    # 출력할 때는 없는 버튼
    # driver.find_element(By.ID, 'btnConfirm').click() # 확인 버튼
    # mwt()

########### 제증명 신청 관련 #################
def apply_certificates(docu_kind : int) :
    "docu_kind : 확정증명:0 | 송달증명:1 | 집행문:3"
    apply_or_print(1) # 1신청, 2발급
    Select(driver.find_element(By.ID, 'cbxDocCd')).select_by_index(docu_kind) #확정증명:0 | 송달증명:1 | 집행문:3
    swt()

def select_parties_new_win() : ##############
    # 양수금, 대여금, 신용카드이용대금 인지
    target_event_name = "양수금|대여금|신용카드이용대금"
    element = wait.until(EC.visibility_of_element_located((By.XPATH, '//*[@id="saNm"]')))
    swt()
    eventname = element.text
    if re.search(target_event_name, element.text) == None : # 타겟 사건명이 아니면
        return False, eventname
    else :
        # 발급당사자 버튼
        wait.until(EC.element_to_be_clickable((By.ID, 'btnSelBalDs'))).click() # 발급당사자선택 버튼
        mwt()

        # 새창 전환
        driver.switch_to.window(driver.window_handles[1]) # 창 전환
        # 팝업창 컨트롤(발급당사자)
        driver.find_element(By.ID, 'allCheck').click() # 당사자 선택
        driver.find_element(By.ID, 'btnReg').click() # 등록
        swt()
        driver.find_element(By.ID, 'selectedAllCheck').click()  # 선택된 당사자 선택
        driver.find_element(By.ID, 'btnSetDs').click() # 확인
        swt()

        try : # 문제가 있다면 alert창이 뜬다. 위의 타겟 사건명도 모두 포함함
            Alert(driver).accept()
            driver.find_element(By.ID, 'btnClose').click() # 취소
            driver.switch_to.window(driver.window_handles[0]) # 창 전환
            return False, eventname
        except (NoSuchWindowException, NoAlertPresentException) : # 알람창 안 떴으니 이게 정상 상태
            # 창 되돌리기
            driver.switch_to.window(driver.window_handles[0]) # 창 전환
            return True, eventname

def check_document() : #############
    time.sleep(7) # pdf문서 대기
    wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="pdfReader"]/iframe')))
    
    # 문서 내용 확인 및 스크롤 (안 되네... )
    # driver.switch_to.frame(driver.find_elements(By.XPATH, '//*[@id="pdfReader"]/iframe')) # 문서viewer iframe 전환
    # driver.execute_script("window.scrollTo(0, document.body.scrollHeight)") # 문서 끝까지 스크롤 
    # driver.find_element(By.XPATH, '//*[@id="body"]/sd-root/sd-main/sd-toolbar/section[3]/section[2]/div/pu-button/div/pu-icon').click()
    
    # 문서 이상없음 체크 및 확인버튼
    driver.find_element(By.ID, 'guidcheck').click() # 문서이상없음
    driver.find_element(By.ID, 'btnNextPage').click() # 확인
    mwt()

def send_confirm() :
    # 제출 및 확인창
    wait.until(EC.element_to_be_clickable((By.ID, 'btnSendJchul'))).click() # 제출 버튼
    swt()
    Alert(driver).accept() # 확인alert
    mwt()
    
########## print ############
def confirm_print() :   # 게시물에 맞는 발급버튼 누르기
    while(True) :
        win_print = au.getWindowsWithTitle("인쇄")
        if win_print == [] :
            time.sleep(3)
        else :
            break
    win_print[0].activate()
    time.sleep(0.2)
    au.press('enter')

def print_execution() :
    "사건번호로 조회 후 집행문 발급"
    kind_docu = "집행문(정본포함)"
    try :
        wait = WebDriverWait(driver, 5) # 기본은 30초로 너무 기니 여기서는 제한한다.
        # kind_docu의 발급 버튼 누르기
        wait.until(EC.element_to_be_clickable((By.XPATH, '//a[@title="'+kind_docu+'"]/parent::td/following-sibling::td[4]/a'))).click()
    except :
        return driver.find_element(By.XPATH, '//a[@title="'+kind_docu+'"]/parent::td/following-sibling::td[4]').text
    wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="tbMain1"]/tbody/tr/td[4]/a'))).click() # 출력버튼
    mwt()
    Alert(driver).accept() # 컨펌창 확인
    time.sleep(7) # 인쇄창 대기 5초에서 7초로 늘림 240514
    confirm_print() # 인쇄창 확인버튼 누르는 함수
    time.sleep(5) 
    Alert(driver).accept() # 컨펌창 확인
    mwt()
    wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="btnPrev"]'))).click() # 이전버튼 
    mwt()
    return "발급"

def print_fix_delivery(kind_docu:str) :
    "사건번호로 조회 후 확정증명, 송달증명 발급"
    # 확정증명 발급. a태그가 없는 경우 td의 text값을 리턴(기발급건은 '발 급'으로 정상적인 '발급'과 구별된다.)
    try :
        wait = WebDriverWait(driver, 5) # 기본은 30초로 너무 기니 여기서는 제한한다.
        wait.until(EC.element_to_be_clickable((By.XPATH, '//a[@title="'+kind_docu+'"]/parent::td/following-sibling::td[4]/a'))).click() # 확정증명의 발급 버튼 누르기
    except :
        return driver.find_element(By.XPATH, '//a[@title="'+kind_docu+'"]/parent::td/following-sibling::td[4]').text
    mwt()
    Alert(driver).accept()
    time.sleep(6)
    # 인쇄 확인 버튼
    confirm_print()
    # 송달증명 발급
    time.sleep(5)
    Alert(driver).accept() # 컨펌창 확인
    mwt()
    return "발급"   # 뒤로 가기 필요 없음

def print_이행권고() : 
    결정문발급버튼 = driver.find_element(By.XPATH, '//*[@id="tbMain"]/tbody/tr[1]/td[5]/a/span')
    if 결정문발급버튼.text == '발 급' : 
        결정문발급버튼.click()
    else : return 결정문발급버튼.text
    mwt()
    Alert(driver).accept()
    time.sleep(6)
    # 인쇄 확인 버튼
    confirm_print()
    # 송달증명 발급
    time.sleep(5)
    Alert(driver).accept() # 컨펌창 확인
    mwt()
    return 결정문발급버튼.text
        

############## 통합 함수 ##############
def fix_delivery(which:int, v) :
    """확정증명 or 송달증명 신청 종합 함수 \n
    which : 확정 - 0, 송달 - 1 \t v : df.iterrows()[1]"""
    apply_certificates(which) # 제증명신청, 제증명 종류
    select_court_event(v.법원, v.사건번호) # 법원 사건번호
    driver.find_element(By.ID, 'btnConfirm').click() # 확인 버튼
    mwt()
    wait.until(EC.element_to_be_clickable((By.ID, "sinGbnCd2"))).click() # 당사자별
    swt()
    select_parties_new_win() # 발급당사자선택 ~ 새창 컨트롤
    driver.find_element(By.ID, "btnComplete").click() # 작성완료
    mwt()
    check_document() # 문서확인
    send_confirm() # 제출 및 comfirm
    certify() # 인증서 
    lwt() # 제출 후 인증은 lwt()추가!
    wait.until(EC.element_to_be_clickable((By.ID, 'btnSendList'))).click() # 완료버튼
    swt()
    # df수정
    if which == 0 : v.확정 = "신청"
    elif which == 1 : v.송달 = "신청"
    
    
def 집행권원_에러시_반복함수(법원, 사건번호) : 
    close_all_popups_and_refresh(driver)
    apply_or_print(2) # 매개변수 2 : 제증명 발급 버튼 클릭
    driver.find_element(By.ID, "selSa").click() # 사건번호 radio
    swt()
    select_court_event(법원, 사건번호)
    driver.find_element(By.ID, "btnRetrieve").click()
    mwt()

### 엑셀 열기 - 키 연결해서 채무상태 확인 후 진행

In [4]:
### 원본이 필요할 땐 원본을 다른 이름으로 복사해둬!
df_origin = pd.read_excel(path_excel, sheet_name="집행문", dtype=dtype).fillna("")
df_origin.법원 = df_origin.법원.str.replace("지법", "지방법원")
df = df_origin.copy()
print(len(df))
df.head(3)

10


,번호,법원,사건번호,재판부,확정일자,원고,피고,순서,집행문,확정,...,비고,가상계좌번호,채무자키,계좌키,법조치키,제소신청일,법조치등록,메모등록,법비용등록,판결문
0,10,광주지방법원,2023가소574793,민사26단독,2024.05.25,주식회사 솔림헬프자산관리대부,황택락,,,,...,,,,,,,,,,
1,4,광주지방법원,2024가단500689,민사 2단독,2024.05.24,주식회사 솔림헬프자산관리대부,김대철,,,,...,,,,,,,,,,
2,5,광주지방법원,2023가소588488,민사22단독,2024.05.23,주식회사 솔림헬프자산관리대부,손현주,,,,...,,,,,,,,,,


### df에 채무자키 계좌키 제소신청일 추가하기

In [5]:
# 최근 dataset pkl 에서 법조치 불러오기
##########################################
last_dataset_day = '240430'
###########################################

fn_law = 파일명(join(r'D:\3.자산\전산 dataset\솔림',last_dataset_day,"pkl"),"법조치조회새창")
df_law = pd.read_pickle(fn_law)

# # 잘 읽어졌나 확인
df_law[df_law["사건번호"]=="2022가소582163"].계좌키.values[0][:9]

'200957567'

In [ ]:
# 구 코드 ###########
# # 법조치조회 업데이트필요하면 전산조회.ipynb에서 작업
# # cols = 채무자키, 매각사구분, 법조치구분, 관할법원, 사건번호, 청구금액법원, 관련사건번호, 매각사
# path_pkl = r'D:\1.전산,개발\3.파일서버관리\0.월별전산조회'
# ##########################################
# file_name = "법조치조회새창_20230621_0914스캔파일작업용"
# ###########################################
# # pkl 읽기
# df_law = pd.read_pickle(join(path_pkl, file_name + ".pkl"))
# # # 잘 읽어졌나 확인
# # df_law[df_law["사건번호"]=="2022가소582163"].계좌키.values[0][:9]

In [6]:
# 키매칭 후 집행문시트에 추가
for i, v in df.iterrows() :
    # 사건번호
    df_result = df_law[(df_law.관할법원== v.법원) & (df_law.사건번호== v.사건번호)].loc[:,["채무자키", "계좌키", "제소신청일", "법조치키", "주민번호"]].fillna("")
    df_result = df_result.sort_values("채무자키") # 다 채무자키인 경우 가장 앞선 채무자키를 부여하기 위해
    df_result = df_result.drop_duplicates(subset="주민번호") # 다 채무자키인 경우에도 하나만 남기기 위함
    
    # 등록 사건번호 있는 경우
    if len(df_result) == 1 : 
        df.loc[i, ["채무자키", "법조치키", "제소신청일"]] = np.concatenate(df_result[["채무자키","법조치키","제소신청일"]].values).tolist() # 제소신청일과 접수일은 차이가 있을 수 있음.. .#######
        df.loc[i, ["계좌키"]] = df_result["계좌키"].values[0][:9] # 다계좌키인경우 하나만 입력하기 위해 
    
    # 등록사건번호 없고, 관련사건번호 있는 경우
    elif len(df_result)  == 0 :
        # 법조치키는 안 불러오기
        df_result_관련 = df_law[((df_law.관련사건법원== v.법원) & (df_law.관련사건번호== v.사건번호)) | ((df_law.관련법조치관할법원== v.법원) & (df_law.관련법조치사건번호== v.사건번호))].loc[:,["채무자키", "계좌키", "제소신청일", "주민번호"]].fillna("")
        df_result_관련 = df_result_관련.sort_values("채무자키")
        df_result_관련 = df_result_관련.drop_duplicates(subset="주민번호")
        
        if len(df_result_관련) == 1 : 
            df.loc[i, ["채무자키", "제소신청일"]] = np.concatenate(df_result_관련[["채무자키","제소신청일"]].values).tolist() # 제소신청일과 접수일은 차이가 있을 수 있음.. .#######
            df.loc[i, ["계좌키"]] = df_result_관련["계좌키"].values[0][:9] # 다계좌키인경우 하나만 입력하기 위해
    
    # 중복 사건 등록 : 법조치 수정할 것    
    else :
        df.loc[i, "채무자키"] = "법조치 중복 등록"
        
df.fillna("", inplace=True)

# 엑셀에 저장
with pd.ExcelWriter(path_excel, engine='openpyxl', mode = 'a', if_sheet_exists = 'replace') as writer:
    df.to_excel(writer, sheet_name = "집행문", index=False)

### 종료건 확인 후 엑셀 다시 열기
- 전산에서 최신 채무상태 및 담당자 확인

In [7]:
# 엑셀 수작업 후 다시 열기
df_origin = pd.read_excel(path_excel, sheet_name="집행문", dtype=dtype).fillna("")
df_origin.법원 = df_origin.법원.str.replace("지법", "지방법원")
df = df_origin.copy()
display(df.head(3))
print("출력할 수 : ",len(df.query('집행문==""'))) # 집행문 신청용이므로 여기서는 권고는 x

,번호,법원,사건번호,재판부,확정일자,원고,피고,순서,집행문,확정,...,비고,가상계좌번호,채무자키,계좌키,법조치키,제소신청일,법조치등록,메모등록,법비용등록,판결문
0,10,광주지방법원,2023가소574793,민사26단독,2024.05.25,주식회사 솔림헬프자산관리대부,황택락,,,,...,,,20429513,200949444,20910043,,,,,
1,4,광주지방법원,2024가단500689,민사 2단독,2024.05.24,주식회사 솔림헬프자산관리대부,김대철,,,,...,,,20499805,201027492,,2024-01-03,,,,
2,5,광주지방법원,2023가소588488,민사22단독,2024.05.23,주식회사 솔림헬프자산관리대부,손현주,,,,...,,,20463684,200983734,20902472,,,,,


출력할 수 :  7


### driver 열기

In [8]:
driver, wait = con_scourt()
login() # 로그인 : 수동으로 하자. 인증서 되는 거 확인

# 집행문 신청

In [9]:
# 대법원 조회
# 오류나면 무한대기 되어버리니 try-except하지말고 그냥 종료시키자.
cnt_success, cnt_fail = 0, 0
for i, v in df.iterrows() :
    try : 
        if v.집행문 == "" :
            # 제증명 신청-집행문 : 3
            apply_certificates(3)
            # 법원 및 사건번호 및 확인
            select_court_event(v.법원, v.사건번호)
            driver.find_element(By.ID, 'btnConfirm').click() # 확인 버튼
            mwt()
            
            # 알람창이 뜨면 문제있는거(ex소취하) 따라서 continue
            try : # 알람창 뜸
                Alert(driver).text
                Alert(driver).dismiss()
                swt()
                df.loc[i, ["집행문","확정","송달"]] = "미대상(알람)"
                cnt_fail += 1
                continue
            except (NoSuchWindowException, NoAlertPresentException) : # 알람창 안 뜸 > 정상
                pass

            # 발급당사자
            if select_parties_new_win()[0] : pass # 양수금 류인경우
            else : 
                df.loc[i, ["집행문","확정","송달"]] = select_parties_new_win()[1] # 양수금이 아닌 경우
                cnt_fail += 1
                continue

            # 동의 및 작성완료
            driver.find_element(By.ID, 'chkAgrElcSo').click() # 동의 체크박스
            driver.find_element(By.ID, 'btnComplete').click() # 작성완료
            mwt()
            Alert(driver).accept() # confirm
            
            # 문서이상없음 및 확인
            check_document() 

            # 은행선택 및 납부
            swt()
            Select(driver.find_element(By.ID, 'vtBankCd')).select_by_visible_text(bank) # 은행 선택
            driver.find_element(By.ID, 'btnElcPay').click() # 납부 버튼
            swt()
            Alert(driver).accept() # confirm
            mwt()
            
            # 제출 및 confirm 확인
            send_confirm()
                
            # 인증서
            certify()
            lwt()

            # 계좌정보 담기 및 완료버튼
            account = driver.find_element(By.ID, 'acno').text # 계좌정보 담기
            wait.until(EC.element_to_be_clickable((By.ID, 'btnSendList'))).click() # 완료버튼
            swt()

            # df 수정 : 계좌번호 및 집행문 상태
            df.loc[i, "가상계좌번호"] = str(account) ########### 형변환 추가
            df.loc[i, "집행문"] = "신청(미납부)"
            cnt_success += 1
    except :
        cnt_fail += 1
        print(f"실패 : {v.사건번호}, {v.피고}")
        continue
        

# 드라이버 종료 및 엑셀 저장
print(f"성공 {cnt_success}건, 실패 {cnt_fail}건")
driver.quit()

with pd.ExcelWriter(path_excel, engine='openpyxl', mode = 'a', if_sheet_exists = 'replace') as writer:
    df.to_excel(writer, sheet_name = "집행문", index=False)

성공 7건, 실패 0건


# 집행문 신청 후(입금 전)

## 이체~판결문목록까지 작성하기

In [10]:
####################################################
cost_day = str(date.today()) # 경우에 따라서 직접 입력 (20230106)
df_valid = df[df.집행문=='신청(미납부)'].copy() # 집행문 발행할 목록
####################################################
cost = 450
print(f'이체건수 : {len(df_valid)}')



# 등록일은 안 써도 됨
# 1) 이체용 df만들기 : 집행문 신청 후 가상계좌번호 받고 난 뒤
dict_cols = {"이체" : ["입금은행", "입금계좌", "고객관리성명", "입금액", "출금통장표시내용", "입금통장표시내용", "입금인코드", "비고", "업체사용key"],
            "법조치전체" : ["법조치키", "채무자키","계좌키","대상자명","담당자","법조치구분","법조치세부","법조치상태","관할법원","사건번호","접수일","법조치메모"],
            "메모" : ["메모키","채무자키","주민번호","매각사","매각사구분","원채권사","메모내용","메모구분","등록일자","등록유저","공지여부","메모색상","약속일자","약속금액","약속구분","약속이행"],
            "법비용" : ["법비용키","채무자키","계좌키","법조치키","법비용일","법비용구분","법비용내역","법비용메모","기타비"],
            "판결문" : ["채무자키","채무자","법원","사건번호","재도_승계_등등"]
            }

content_display=[]
names = []
for i in df_valid.피고.str.replace("주식회사|유한회사|법인|대표|보증인?|\b[주유]\b|\(주|주\)|외\s?\d.*", "", regex=True) :
    
    # 법조치 대상자명용
    i = re.sub("[^가-힣a-zA-Z]", " ", i).strip()
    sign = re.search("\s", i)
    if sign :
        new_i = i[:sign.start()]
    else :
        new_i = i
    if len(new_i) >= 2 : 
        names.append(new_i)
    else : names.append(i)
    # 이체 화면표시용
    if len(new_i) > 3 : content = new_i[:3]+" 제증명"
    elif len(new_i) >= 2 :
        content = new_i + " 제증명"
    elif len(i) > 3 : 
        content = i[:3] + " 제증명"
    else :
        content = i + " 제증명"
    content_display.append(content)

df_transfer = pd.DataFrame(None, columns=dict_cols["이체"])
df_transfer["입금계좌"] = df_valid.가상계좌번호
df_transfer["출금통장표시내용"] = content_display
# 단일 값은 뒤에(그래야 모두 채워지니)
df_transfer["입금은행"] = "088"
df_transfer["입금액"] = cost

# 4) 법비용 #법조치키는 전산에서 조회해서 수동update
######## 수정일, 수정유저(나) 검색 셀추가 및 필터링 ########
######## 검색 결과 정렬이 바뀌니, 엑셀파일과 검색결과의 정렬 맞춘 후 복붙 ########
df_cost = pd.DataFrame(None, columns=dict_cols["법비용"])
df_cost["채무자키"] = df_valid.채무자키
df_cost["계좌키"] = df_valid.계좌키
df_cost["법조치키"] = df_valid.법조치키
df_cost["법비용일"] = cost_day
df_cost["법비용구분"] = "기본비용"
df_cost["법비용내역"] = "초도부여-신청비용" # 집행문부여
df_cost["집행문발급비"] = cost


# df_valid 재작성 : 이행권고 포함 -------------------------------------------------------------------------------------------------
if 이행권고출력 : 
    df_valid2 = df[df.집행문.str.contains("신청|이행권고")].copy() # 집행문 발행할 목록 ##################################
else : df_valid2 = df_valid

names = []
for i in df_valid2.피고.str.replace("주식회사|유한회사|법인|대표|보증인?|\b[주유]\b|\(주|주\)|외\s?\d.*", "", regex=True) :
    
    # 법조치 대상자명용
    i = re.sub("[^가-힣a-zA-Z]", " ", i).strip()
    sign = re.search("\s", i)
    if sign :
        new_i = i[:sign.start()]
    else :
        new_i = i
    if len(new_i) >= 2 : 
        names.append(new_i)
    else : names.append(i)
    

# 2) 법조치 : df에 채무자키 등 추가 후
df_legalaction = pd.DataFrame(None, columns=dict_cols["법조치전체"])
df_legalaction["법조치키"] = df_valid2.법조치키
df_legalaction["채무자키"] = df_valid2.채무자키
df_legalaction["계좌키"] = df_valid2.계좌키
df_legalaction["대상자명"] = names
df_legalaction["관할법원"] = df_valid2.법원
df_legalaction["사건번호"] = df_valid2.사건번호
df_legalaction["접수일"] = df_valid2.제소신청일
df_legalaction["법조치구분"] = "본안소송"
df_legalaction["법조치세부"] = "전자소송"
df_legalaction["법조치상태"] = "법조치완료"

# 3) 메모
date.today()
df_memo = pd.DataFrame(None, columns=dict_cols["메모"])
df_memo["채무자키"] = df_valid2.채무자키
df_memo["메모내용"] = df_valid2.apply(lambda row: f"{row['사건번호']} 집행문 발급(이행권고)" if row["집행문"] == "이행권고" else f"{row['사건번호']} 집행문 발급", axis=1)
df_memo["등록일자"] = str(date.today()) # str(date.today() + timedelta(days=3)) # 금요일에 작업하고, 월요일에 등록하는 경우 
df_memo["메모구분"] = "기타"
df_memo["등록유저"] = "전병만"

# 5) 판결문
df_list = pd.DataFrame(None, columns=dict_cols["판결문"])
df_list["채무자키"] = df_valid2.채무자키
df_list["채무자"] = df_valid2.피고
df_list["법원"] = df_valid2.법원
df_list["사건번호"] = df_valid2.사건번호

print(f'사건건수 : {len(df_valid2)}')

이체건수 : 7
사건건수 : 7


#### 엑셀 작성

In [11]:
dict_sheet_df = {"이체": df_transfer, "법조치전체":df_legalaction,"메모":df_memo,"법비용":df_cost,"판결문":df_list}

with pd.ExcelWriter(path_excel, engine='openpyxl', mode = 'a', if_sheet_exists = 'replace') as writer:
    for sheetname, df_name in dict_sheet_df.items():
        df_name.to_excel(writer, sheet_name = sheetname, index=False)

# 입금
- 만료일 10-27

# 확정증명원/송달증명원 신청

In [12]:
driver, wait = con_scourt()
login() # 로그인 : 수동으로 하자. 인증서 되는 거 확인

In [15]:
### 원본이 필요할 땐 원본을 다른 이름으로 복사해둬!
df_origin = pd.read_excel(path_excel, sheet_name="집행문", dtype=dtype).fillna("")
df_origin.법원 = df_origin.법원.str.replace("지법", "지방법원")
df = df_origin.copy()
print("신청할 사건 수 : ", len(df.query('집행문.str.contains("신청|발급")')))
df.query('집행문.str.contains("신청|발급")').head(3)

신청할 사건 수 :  7


,번호,법원,사건번호,재판부,확정일자,원고,피고,순서,집행문,확정,...,비고,가상계좌번호,채무자키,계좌키,법조치키,제소신청일,법조치등록,메모등록,법비용등록,판결문
0,10,광주지방법원,2023가소574793,민사26단독,2024.05.25,주식회사 솔림헬프자산관리대부,황택락,,신청(미납부),,...,,56216730504393,20429513,200949444,20910043,,,,,
1,4,광주지방법원,2024가단500689,민사 2단독,2024.05.24,주식회사 솔림헬프자산관리대부,김대철,,신청(미납부),신청,...,,56216730504444,20499805,201027492,,2024-01-03,,,,
2,5,광주지방법원,2023가소588488,민사22단독,2024.05.23,주식회사 솔림헬프자산관리대부,손현주,,신청(미납부),신청,...,,56216730504459,20463684,200983734,20902472,,,,,


In [16]:
# 건당 1분씩 소요
cnt_success_fix, cnt_success_del, cnt_fail = 0, 0, 0
for i, v in df.iterrows() :
    if v.집행문.find('신청') >= 0 :
        try :
            if v.송달 == "" :
                fix_delivery(1, v)
                df.loc[i, "송달"] = "신청" # df수정
                cnt_success_del += 1

            if v.확정 == "" :
                fix_delivery(0, v)
                df.loc[i, "확정"] = "신청" # df수정
                cnt_success_fix += 1
        except :
            cnt_fail += 1

print(f"신청성공[확정증명원 : {cnt_success_fix}건, 송달증명원 : {cnt_success_del}건],    실패 {cnt_fail}건")
# 엑셀 저장
with pd.ExcelWriter(path_excel, engine='openpyxl', mode = 'a', if_sheet_exists = 'replace') as writer:
    df.to_excel(writer, sheet_name = "집행문", index=False)

신청성공[확정증명원 : 2건, 송달증명원 : 1건],    실패 0건


# 출력

##### 엑셀 및 드라이버 열기

In [3]:
### 원본이 필요할 땐 원본을 다른 이름으로 복사해둬!
df_origin = pd.read_excel(path_excel, sheet_name="집행문", dtype=dtype).fillna("")
df_origin.법원 = df_origin.법원.str.replace("지법", "지방법원")
df = df_origin.copy()
print(len(df))
df.query('집행문.str.contains("신청|발급|이행권고")').head(3)

10


,번호,법원,사건번호,재판부,확정일자,원고,피고,순서,집행문,확정,...,비고,가상계좌번호,채무자키,계좌키,법조치키,제소신청일,법조치등록,메모등록,법비용등록,판결문
0,10,광주지방법원,2023가소574793,민사26단독,2024.05.25,주식회사 솔림헬프자산관리대부,황택락,,신청(미납부),신청,...,,56216730504393,20429513,200949444,20910043,,,,,
1,4,광주지방법원,2024가단500689,민사 2단독,2024.05.24,주식회사 솔림헬프자산관리대부,김대철,,신청(미납부),신청,...,,56216730504444,20499805,201027492,,2024-01-03,,,,
2,5,광주지방법원,2023가소588488,민사22단독,2024.05.23,주식회사 솔림헬프자산관리대부,손현주,,신청(미납부),신청,...,,56216730504459,20463684,200983734,20902472,,,,,


In [4]:
driver, wait = con_scourt()
login() # 로그인 : 수동으로 하자. 인증서 되는 거 확인

##### 프리턴 설정 확인 not pdf, 이행권고 미확인 문서 일괄확인할 것

In [5]:
class CustomBreakException(Exception): # 내for문에서라도 해당 지점에서 모든 반복문을 종료하기 위한 클래스(break)
    pass

cnt_success, cnt_success_fix, cnt_success_del, cnt_success_권고 = 0,0,0,0
cnt_fail, cnt_fail_fix, cnt_fail_del, cnt_fail_권고 = 0,0,0,0

try : 
    for i, v in df.iterrows() :
        ###################
        #if v.번호 < 2 :
        ###################
        # 1) 로그인 후 발급할 것이 있는 사건번호를 검색(조회)
        if (re.search("신청",v.집행문)) or (v.확정 == "신청") or (v.송달 == "신청") : 
            
            temp_cnt = 0
            max_attempts = 3
            for attempt in range(max_attempts) : # 사건조회에서 에러나면 
                try : 
                    # 사건번호로 조회
                    apply_or_print(2) # 매개변수 2 : 제증명 발급 버튼 클릭
                    driver.find_element(By.ID, "selSa").click() # 사건번호 radio
                    swt()
                    select_court_event(v.법원, v.사건번호)
                    driver.find_element(By.ID, "btnRetrieve").click()
                    mwt()
                    
                except Exception as e :
                    temp_cnt += 1
                    if attempt < max_attempts :
                        print(f"조회화면 에러 재시도 : {temp_cnt}회")
                    else:
                        print("최대 시도 횟수를 초과하여 모든 작업 중단")
                        all_break = True
                    
                    # 오류시 모든 팝업 닫고 새로고침
                    close_all_popups_and_refresh(driver)
                        
                
            # 2) 해당하는 것을 모두 출력(발급) - 집행문 에러나도 확정, 송달 출력되게 하나씩 예외처리
            try : 
                if re.search("신청",v.집행문) : # v.집행문은 str임... 미납부 > 납부 수정 깜박하는 경우가 많아서.. '==신청(납부)'에서 바꿈
                    exe_result = print_execution() # 집행문(정본포함) 하나이므로 매개변수 없이 지역변수로 선언함
                    df.loc[i, "집행문"] = exe_result
                    if exe_result == "발급" : cnt_success += 1
            except : 
                print(v.사건번호, "집행문 출력 중 에러")
                cnt_fail += 1
                
                집행권원_에러시_반복함수(v.법원, v.사건번호) ################### 에러 나는지 확인 #################

            try : 
                if v.송달 == "신청" : # 송달도 발급해야 하면 연속 발급
                    del_result = print_fix_delivery("송달증명")
                    df.loc[i, "송달"] = del_result
                    if del_result == "발급" : cnt_success_del += 1
            except : 
                print(v.사건번호, "송달증명 출력 중 에러")
                cnt_fail_del += 1
                
                집행권원_에러시_반복함수(v.법원, v.사건번호) ################### 에러 나는지 확인 #################

            try : 
                if v.확정 == "신청" :
                    fix_result = print_fix_delivery("확정증명")
                    df.loc[i, "확정"] = fix_result
                    if fix_result == "발급" : cnt_success_fix += 1
            except : 
                print(v.사건번호, "확정증명 출력 중 에러")
                cnt_fail_fix += 1
                
                close_all_popups_and_refresh(driver) # 마지막이라 사건번호 입력할 필요는 없음.
                    
        elif re.match("이행권고", v.집행문) and 이행권고출력 : # 이행권고 출력일때만
            try : 
                # 발급조회
                이행권고()
                driver.find_element(By.ID, "selSa").click() # 사건번호 radio
                swt()
                select_court_event(v.법원, v.사건번호)
                driver.find_element(By.ID, "btnRetrieve").click()
                mwt()
                
                # 출력
                권고_result = re.sub(" ", "", print_이행권고()) # 공백 제거
                df.loc[i, "집행문"] = 권고_result + "_" + v.집행문
                if 권고_result == '발급' : cnt_success_권고 += 1
            except : 
                print(v.사건번호, "이행권고 출력 중 에러")
                cnt_fail_권고 += 1
        else : 
            pass
            
except CustomBreakException:
    print("에러로 모든 반복문 종료")
    
print(f"출력성공[집행문:{cnt_success}건, 확정:{cnt_success_fix}건, 송달:{cnt_success_del}건, 이행권고:{cnt_success_권고}건]")
print(f"출력실패[집행문:{cnt_fail}건, 확정:{cnt_fail_fix}건, 송달:{cnt_fail_del}건, 이행권고:{cnt_fail_권고}건]")
# 드라이버 종료 및 엑셀 저장
driver.quit()
# 엑셀 저장
with pd.ExcelWriter(path_excel, engine='openpyxl', mode = 'a', if_sheet_exists = 'replace') as writer:
    df.to_excel(writer, sheet_name = "집행문", index=False)

# 이행권고 기발급건(재발급요청)은 메모와 판결문 목록에서는 삭제해줘야

2024가단500689 집행문 출력 중 에러
출력성공[집행문:6건, 확정:7건, 송달:7건, 이행권고:0건]
출력실패[집행문:1건, 확정:0건, 송달:0건, 이행권고:0건]


# 발급 및 스캔 후 : 스캔파일
- 인쇄 순서 엉킨 경우에는 순서열 만들어서 정렬 후 읽기

In [6]:
# 스캔파일 목록을 df_scan에 추가하기
### 원본이 필요할 땐 원본을 다른 이름으로 복사해둬!
df_origin = pd.read_excel(path_excel, sheet_name="집행문", dtype=dtype).fillna("")
df = df_origin.copy()
df_valid = df[df.집행문.str.match('발급')].copy().reset_index(drop=True) # 실제 작업할 영역 불러오기 # 240415 이행권고 때문에 =='발급'에서 포함으로 바꿈
# df_valid = df.iloc[:10,:].copy() # 실제 작업할 영역 불러오기
##################################################
path = r"D:\scan"
file_list = read_filelist(path) # 탐색기 정렬과 무관하게 이름 오름차순으로 정렬된 리스트 반환
df_valid["file"] = file_list

In [7]:
# df_valid 한행씩 불러와서 파일명 바꾸기
success, fail = [], []
for i, v in df_valid.iterrows() :
    
    ext = os.path.splitext(v.file)[1]
    if re.search("이행권고", v.집행문) : 
        new_name = "_".join([str(int(v.채무자키)), v.피고, v.사건번호, "이행권고"]) + ext
    else : 
        new_name = "_".join([str(int(v.채무자키)), v.피고, v.사건번호]) + ext
    try :
        os.rename(join(path, v.file), join(path, new_name))
        success.append([path, v.file, new_name])
    except Exception as e :
        fail.append([path, v.file, new_name, e.__class__])
        continue
print(f"총 {len(df_valid)}개 중, {len(success)}개 파일명 변경, {len(fail)}개 파일명 변경 실패")

총 7개 중, 7개 파일명 변경, 0개 파일명 변경 실패


In [ ]:
# 옛날꺼
# path = r"D:\scan\새 폴더"
# p_e = re.compile("\d{4}[가-힣]{1,3}\d+")
# df_result = pd.DataFrame(columns=["채무자키", "계좌키", "사건번호", "계좌키", "제소신청일"])
# for f in os.listdir(path) :
#     stem, ext = os.path.splitext(f)
#     e_name = p_e.search(stem).group()
#     # drop_duplicates(subset=["채무자키"], keep='first').
#     keys = df_law[(df_law.관련사건번호== e_name) |(df_law.사건번호== e_name)].loc[:,["채무자키", "계좌키", "사건번호", "계좌키", "제소신청일"]]
#     df_result = pd.concat([df_result, keys])
    
# df_result    

#### 법조치 등록 후 법비용에 법조치키 수동 입력
- 법조치새창 > 사건번호 멀티검색 > "법조치시트"에서 사건번호 가져와 > 수정유저 컬럼 추가 후 '자신'으로 필터링
- 순서 다르므로 엑셀과 전산프로그램 둘다 채무자키로 정렬 후 "법비용시트"에 복붙